# ClickGraph Relationship Testing

This notebook tests the relationship functionality in ClickGraph using the social network view configuration.

## Configuration Overview

The `examples/social_network_view.yaml` defines:
- **Nodes**: user, post, customer, product
- **Relationships**: AUTHORED, LIKED, FOLLOWS, PURCHASED

Server startup shows:
```
- Loaded 4 relationship types: ["AUTHORED", "LIKED", "FOLLOWS", "PURCHASED"]
```

Our goal is to test if these relationship types can generate proper JOIN SQL queries.

## Test 1: AUTHORED Relationship (User → Post)

The AUTHORED relationship connects users to posts they have written.

In [5]:
import requests
import json

def test_cypher_query(query, description, sql_only=True):
    """Test a Cypher query and return the result"""
    url = "http://localhost:8081/query"  # Fixed endpoint
    headers = {"Content-Type": "application/json"}
    data = {"query": query, "sql_only": sql_only}  # SQL-only mode to avoid ClickHouse execution
    
    print(f"\n{description}")
    print(f"Query: {query}")
    print("-" * 80)
    
    try:
        response = requests.post(url, headers=headers, json=data)
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print(f"Response: {json.dumps(result, indent=2)}")
            return True, result
        else:
            print(f"Error Response: {response.text}")
            return False, None
            
    except Exception as e:
        print(f"Connection Error: {e}")
        return False, None

# Test AUTHORED relationship
query = "MATCH (u:user)-[r:AUTHORED]->(p:post) RETURN u.name, p.title LIMIT 5"
test_cypher_query(query, "Testing AUTHORED relationship between users and posts")


Testing AUTHORED relationship between users and posts
Query: MATCH (u:user)-[r:AUTHORED]->(p:post) RETURN u.name, p.title LIMIT 5
--------------------------------------------------------------------------------
Status Code: 200
Response: {
  "cypher_query": "MATCH (u:user)-[r:AUTHORED]->(p:post) RETURN u.name, p.title LIMIT 5",
  "generated_sql": "WITH user_u AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\n), \nAUTHORED_r AS (\n    SELECT \n      u.from_user AS from_id, \n      p.to_post AS to_id\nFROM AUTHORED AS t\nWHERE t.to_id IN (SELECT u.user_id FROM user_u AS t)\n)\nSELECT \n      u.name, \n      p.title\nFROM post AS p\nINNER JOIN AUTHORED_r AS r ON r.from_id = p.post_id\nINNER JOIN user_u AS u ON u.user_id = r.to_id\nLIMIT  5",
  "execution_mode": "sql_only"
}
Status Code: 200
Response: {
  "cypher_query": "MATCH (u:user)-[r:AUTHORED]->(p:post) RETURN u.name, p.title LIMIT 5",
  "generated_sql": "WITH user_u AS (\n    SELECT \n      u.name, \n      u.user_

(True,
 {'cypher_query': 'MATCH (u:user)-[r:AUTHORED]->(p:post) RETURN u.name, p.title LIMIT 5',
  'generated_sql': 'WITH user_u AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\n), \nAUTHORED_r AS (\n    SELECT \n      u.from_user AS from_id, \n      p.to_post AS to_id\nFROM AUTHORED AS t\nWHERE t.to_id IN (SELECT u.user_id FROM user_u AS t)\n)\nSELECT \n      u.name, \n      p.title\nFROM post AS p\nINNER JOIN AUTHORED_r AS r ON r.from_id = p.post_id\nINNER JOIN user_u AS u ON u.user_id = r.to_id\nLIMIT  5',
  'execution_mode': 'sql_only'})

## Test 2: FOLLOWS Relationship (User → User)

The FOLLOWS relationship represents user-to-user following connections in a social network.

In [6]:
# Test FOLLOWS relationship
query = "MATCH (u1:user)-[r:FOLLOWS]->(u2:user) RETURN u1.name AS follower, u2.name AS following LIMIT 5"
test_cypher_query(query, "Testing FOLLOWS relationship between users")


Testing FOLLOWS relationship between users
Query: MATCH (u1:user)-[r:FOLLOWS]->(u2:user) RETURN u1.name AS follower, u2.name AS following LIMIT 5
--------------------------------------------------------------------------------
Status Code: 200
Response: {
  "cypher_query": "MATCH (u1:user)-[r:FOLLOWS]->(u2:user) RETURN u1.name AS follower, u2.name AS following LIMIT 5",
  "generated_sql": "WITH user_u1 AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\n), \nFOLLOWS_r AS (\n    SELECT \n      u.from_user AS from_id, \n      u.to_user AS to_id\nFROM FOLLOWS AS t\nWHERE t.to_id IN (SELECT u.user_id FROM user_u1 AS t)\n)\nSELECT \n      u1.name AS follower, \n      u2.name AS following\nFROM user AS u2\nINNER JOIN FOLLOWS_r AS r ON r.to_id = u2.user_id\nINNER JOIN user_u1 AS u1 ON u1.user_id = r.from_id\nLIMIT  5",
  "execution_mode": "sql_only"
}


(True,
 {'cypher_query': 'MATCH (u1:user)-[r:FOLLOWS]->(u2:user) RETURN u1.name AS follower, u2.name AS following LIMIT 5',
  'generated_sql': 'WITH user_u1 AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\n), \nFOLLOWS_r AS (\n    SELECT \n      u.from_user AS from_id, \n      u.to_user AS to_id\nFROM FOLLOWS AS t\nWHERE t.to_id IN (SELECT u.user_id FROM user_u1 AS t)\n)\nSELECT \n      u1.name AS follower, \n      u2.name AS following\nFROM user AS u2\nINNER JOIN FOLLOWS_r AS r ON r.to_id = u2.user_id\nINNER JOIN user_u1 AS u1 ON u1.user_id = r.from_id\nLIMIT  5',
  'execution_mode': 'sql_only'})

## Test 3: LIKED Relationship (User → Post)

The LIKED relationship represents users liking posts.

In [7]:
# Test LIKED relationship
query = "MATCH (u:user)-[r:LIKED]->(p:post) RETURN u.name AS liker, p.title AS liked_post LIMIT 5"
test_cypher_query(query, "Testing LIKED relationship between users and posts")


Testing LIKED relationship between users and posts
Query: MATCH (u:user)-[r:LIKED]->(p:post) RETURN u.name AS liker, p.title AS liked_post LIMIT 5
--------------------------------------------------------------------------------
Status Code: 200
Response: {
  "cypher_query": "MATCH (u:user)-[r:LIKED]->(p:post) RETURN u.name AS liker, p.title AS liked_post LIMIT 5",
  "generated_sql": "WITH user_u AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\n), \nLIKED_r AS (\n    SELECT \n      u.from_user AS from_id, \n      p.to_post AS to_id\nFROM LIKED AS t\nWHERE t.to_id IN (SELECT u.user_id FROM user_u AS t)\n)\nSELECT \n      u.name AS liker, \n      p.title AS liked_post\nFROM post AS p\nINNER JOIN LIKED_r AS r ON r.from_id = p.post_id\nINNER JOIN user_u AS u ON u.user_id = r.to_id\nLIMIT  5",
  "execution_mode": "sql_only"
}
Status Code: 200
Response: {
  "cypher_query": "MATCH (u:user)-[r:LIKED]->(p:post) RETURN u.name AS liker, p.title AS liked_post LIMIT 5",
  "genera

(True,
 {'cypher_query': 'MATCH (u:user)-[r:LIKED]->(p:post) RETURN u.name AS liker, p.title AS liked_post LIMIT 5',
  'generated_sql': 'WITH user_u AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\n), \nLIKED_r AS (\n    SELECT \n      u.from_user AS from_id, \n      p.to_post AS to_id\nFROM LIKED AS t\nWHERE t.to_id IN (SELECT u.user_id FROM user_u AS t)\n)\nSELECT \n      u.name AS liker, \n      p.title AS liked_post\nFROM post AS p\nINNER JOIN LIKED_r AS r ON r.from_id = p.post_id\nINNER JOIN user_u AS u ON u.user_id = r.to_id\nLIMIT  5',
  'execution_mode': 'sql_only'})

## Test 4: PURCHASED Relationship (Customer → Product)

The PURCHASED relationship connects customers to products they have bought.

In [8]:
# Test PURCHASED relationship
query = "MATCH (c:customer)-[r:PURCHASED]->(p:product) RETURN c.name AS customer, p.name AS product LIMIT 5"
test_cypher_query(query, "Testing PURCHASED relationship between customers and products")


Testing PURCHASED relationship between customers and products
Query: MATCH (c:customer)-[r:PURCHASED]->(p:product) RETURN c.name AS customer, p.name AS product LIMIT 5
--------------------------------------------------------------------------------
Status Code: 200
Response: {
  "cypher_query": "MATCH (c:customer)-[r:PURCHASED]->(p:product) RETURN c.name AS customer, p.name AS product LIMIT 5",
  "generated_sql": "WITH customer_c AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM customer AS t\n), \nPURCHASED_r AS (\n    SELECT \n      c.from_customer AS from_id, \n      product.to_product AS to_id\nFROM PURCHASED AS t\nWHERE t.to_id IN (SELECT u.user_id FROM customer_c AS t)\n)\nSELECT \n      c.name AS customer, \n      p.name AS product\nFROM product AS p\nINNER JOIN PURCHASED_r AS r ON r.from_id = p.product_id\nINNER JOIN customer_c AS c ON c.user_id = r.to_id\nLIMIT  5",
  "execution_mode": "sql_only"
}
Status Code: 200
Response: {
  "cypher_query": "MATCH (c:customer)-[r:P

(True,
 {'cypher_query': 'MATCH (c:customer)-[r:PURCHASED]->(p:product) RETURN c.name AS customer, p.name AS product LIMIT 5',
  'generated_sql': 'WITH customer_c AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM customer AS t\n), \nPURCHASED_r AS (\n    SELECT \n      c.from_customer AS from_id, \n      product.to_product AS to_id\nFROM PURCHASED AS t\nWHERE t.to_id IN (SELECT u.user_id FROM customer_c AS t)\n)\nSELECT \n      c.name AS customer, \n      p.name AS product\nFROM product AS p\nINNER JOIN PURCHASED_r AS r ON r.from_id = p.product_id\nINNER JOIN customer_c AS c ON c.user_id = r.to_id\nLIMIT  5',
  'execution_mode': 'sql_only'})

## Test 5: Complex Multi-Hop Queries

Testing complex queries that traverse multiple relationships.

In [3]:
# Test multi-hop query: Find posts liked by followers of a user
query = """MATCH (u:user)-[f:FOLLOWS]->(follower:user)-[l:LIKED]->(p:post) 
           RETURN u.name AS user, follower.name AS follower, p.title AS liked_post LIMIT 5"""
test_cypher_query(query, "Testing multi-hop query: user -> follows -> liked posts")

# Test query with relationship properties (if supported)
query2 = "MATCH (u:user)-[r:AUTHORED {published: true}]->(p:post) RETURN u.name, p.title LIMIT 3"
test_cypher_query(query2, "Testing relationship with properties (if supported)")


Testing multi-hop query: user -> follows -> liked posts
Query: MATCH (u:user)-[f:FOLLOWS]->(follower:user)-[l:LIKED]->(p:post) 
           RETURN u.name AS user, follower.name AS follower, p.title AS liked_post LIMIT 5
--------------------------------------------------------------------------------
Status Code: 404
Error Response: 

Testing relationship with properties (if supported)
Query: MATCH (u:user)-[r:AUTHORED {published: true}]->(p:post) RETURN u.name, p.title LIMIT 3
--------------------------------------------------------------------------------
Status Code: 404
Error Response: 

Testing relationship with properties (if supported)
Query: MATCH (u:user)-[r:AUTHORED {published: true}]->(p:post) RETURN u.name, p.title LIMIT 3
--------------------------------------------------------------------------------
Status Code: 404
Error Response: 
Status Code: 404
Error Response: 


(False, None)